<a href="https://colab.research.google.com/github/brandonso994/AttnLSTMMusicGeneration/blob/main/V4/Generate_Notes_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras_self_attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras_self_attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=a1654c70a56d482084578473189081c051012e4a164b1f71358eeaeca88d874d
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras_self_attention


In [ ]:
from music21 import converter, instrument, note, chord, stream, volume
from fractions import Fraction
import matplotlib.pyplot as plt
import glob
import numpy as np
import nltk
import pandas as pd
import pickle
from keras.models import Sequential, Model
from keras.layers import LSTM, Dropout, Dense, Activation, Bidirectional, BatchNormalization, Input, Embedding, Concatenate, Flatten
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import CategoricalAccuracy
import tensorflow as tf
from keras import backend as K
from keras_self_attention import SeqSelfAttention

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def flatten(array):
  new_array = [item for array in array for item in array]
  return new_array

# Create Sequence with window length = sequence_length for input into model
def sequence(notes ,full_notes, note_set_len, sequence_length, midi_input):

  note_values = sorted(set(note for note in full_notes))
  note_encode = dict((note, num) for num, note in enumerate(note_values))

  network_input_note = []
  note_output = []

  for i in range (len(notes) - sequence_length):
    sequence_in_note = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input_note.append([note_encode[note] for note in sequence_in_note])
    note_output.append(note_encode[sequence_out])
    if midi_input:
      break

  n_patterns = len(network_input_note)

  network_input_note_norm = np.reshape(network_input_note, (n_patterns, sequence_length, 1))
  network_input_note_norm = network_input_note_norm / float(note_set_len)

  note_output = np_utils.to_categorical(note_output)

  return network_input_note, network_input_note_norm

def create_model(network_input_note, note_set_len, version_num):

    model = Sequential()

    # Note input
    model.add(Bidirectional(LSTM(512,return_sequences=True),input_shape=(network_input_note.shape[1], network_input_note.shape[2])))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    model.add(LSTM(512,return_sequences=False))
    model.add(Dropout(0.3))

    model.add(Dense(note_set_len, activation='softmax', name='note'))
    model.load_weights('/content/drive/My Drive/MRP/Model/model_weights_' + version_num + '_checkpoints.h5')

    return model

# Predicts the next note given the sequence
def note_prediction(model, note_input, notes, seq_len, num_notes=500, midi_input=False, temp=0.8):
  note_set_len = len(set(notes))

  note_values = sorted(set(note for note in notes))

  note_decode = dict((num, note) for num, note in enumerate(note_values))
  note_encode = dict((note, num) for num, note in enumerate(note_values))

  # Whether input was given, or start randomly from sequence
  if midi_input:
    start = 0
  else:
    start = np.random.randint(0, len(note_input) - 1)

  generated_notes = []
  note_pattern = note_input[start]

  # Loop, predicting each note and appending the prediction to the sequence, removing the oldest note to preserve sequence length
  for item in range(num_notes):
    prediction_input = np.reshape(note_pattern, (1, len(note_pattern), 1))
    prediction_input = prediction_input / float(note_set_len)

    predictions = model.predict(prediction_input, verbose=0)

    predicted_note = predictions

    # Apply temperature
    predicted_note = np.log(predicted_note) / temp

    note_probs = np.exp(predicted_note) / np.sum(np.exp(predicted_note))

    # Sample the next pitch and duration using the temperature-adjusted probabilities
    next_note = np.random.choice(len(note_probs[0]), p=note_probs[0])

    decoded_note = note_decode[next_note]
    generated_notes.append(decoded_note)
    note_pattern.append(next_note)
    note_pattern = note_pattern[1:len(note_pattern)]

    # Decrease temperature slowly if greater than 1
    if temp > 1:
      temp -= 0.2


  return generated_notes

# Generate midi from predicted notes
def generate_midi(note_sequence, file_num, version_num):
  output_notes = []
  offset = 0
  isRest = False
  for output_note in note_sequence:
      output_note = output_note.split()
      temp = output_note[0]
      duration = float(output_note[1].split(':')[0])
      new_offset = float(output_note[1].split(':')[1])
      output_pitch = temp
      # If chord
      if ('.' in output_pitch) or output_pitch.isdigit():
          notes_in_chord = output_pitch.split('.')
          notes = []
          for current_note in notes_in_chord:
              new_note = note.Note(int(current_note))
              notes.append(new_note)
          new_chord = chord.Chord(notes)
          new_chord.duration.quarterLength = float(duration)
          new_chord.offset = offset
          output_notes.append(new_chord)
      else:
          new_note = note.Note(output_pitch)
          new_note.offset = offset
          new_note.duration.quarterLength = float(duration)
          output_notes.append(new_note)
      offset += new_offset

  midi_stream = stream.Stream(output_notes)
  piano = instrument.Piano()
  midi_stream.insert(0, piano)
  midi_stream.write('midi', fp='/content/drive/My Drive/MRP/test_file_' + str(version_num) + '_' + str(file_num) + '.midi')

  return

def generate(version_num, file_num, midi_input=False, load=False, num_notes=500):
  seq_len = 60

  if midi_input:
    with open('simple_notes_offsets_midi.pkl', 'rb') as f:
      midi_note = pickle.load(f)

    midi_note = flatten(midi_note)

  with open('simple_notes_offsets.pkl', 'rb') as f:
    notes = pickle.load(f)

  notes = flatten(notes)
  note_set_len = len(set(notes))

  if midi_input:
    print(midi_input)
    note_input, note_input_norm = sequence(midi_note, notes, note_set_len, seq_len, midi_input)
  else:
    note_input, note_input_norm = sequence(notes, notes, note_set_len, seq_len, midi_input)

  model = create_model(note_input_norm, note_set_len, version_num)
  generated_notes = note_prediction(model, note_input, notes, seq_len, num_notes, midi_input=midi_input)

  generate_midi(generated_notes, file_num, version_num)

  return





In [ ]:
for x in range(5,11):
  generate('v4_3', x)

In [ ]:
for x in range (1,4):
  generate("v4_4_2", "Hayden" + "_" + str(x), midi_input=True, num_notes = 300)

True
True
True
